In [202]:
exec(open("/users/madke/documents/092520_rfregressorhelperfunctions.py").read())
from math import acos

In [219]:
shuffle_clean = pd.read_csv('/users/madke/documents/012320AAPECS_shuffleclean.csv', index_col = 0)
shuffle = pd.read_csv('/users/madke/documents/012320AAPECS_shuffle.csv', index_col = 0)
consec_clean = pd.read_csv('/users/madke/documents/012320AAPECS_rigidityclean.csv', index_col = 0)
consec = pd.read_csv('/users/madke/documents/012320AAPECS_rigidity.csv', index_col = 0)
dims = pd.read_csv('/users/madke/documents/090820_aapecsdims4.csv', index_col = 0)
formal_vals = pd.read_csv('/users/madke/documents/092520_aapecsformalvals.csv', index_col = 0)

In [236]:
shuffle = shuffle.replace(1, np.NaN)
consec = consec.replace(1,np.NaN)

consec['pwvDmed'] = [acos(i) for i in list(consec['pwvDmed'])]
consec['swvDmed'] = [acos(i) for i in list(consec['swvDmed'])]
consec['uwvDmed'] = [acos(i) for i in list(consec['uwvDmed'])]


consec_clean['pwvDmed'] = [acos(i) for i in list(consec_clean['pwvDmed'])]
consec_clean['swvDmed'] = [acos(i) for i in list(consec_clean['swvDmed'])]
consec_clean['uwvDmed'] = [acos(i) for i in list(consec_clean['uwvDmed'])]
consec_clean = consec_clean.rename(columns=lambda x: 'clean_' + x)

shuffle['pwvVar'] = [acos(i) for i in list(shuffle['pwvVar'])]
shuffle['swvVar'] = [acos(i) for i in list(shuffle['swvVar'])]
shuffle['uwvVar'] = [acos(i) for i in list(shuffle['uwvVar'])]


shuffle_clean['pwvVar'] = [acos(i) for i in list(shuffle_clean['pwvVar'])]
shuffle_clean['swvVar'] = [acos(i) for i in list(shuffle_clean['swvVar'])]
shuffle_clean['uwvVar'] = [acos(i) for i in list(shuffle_clean['uwvVar'])]
shuffle_clean = shuffle_clean.rename(columns=lambda x: 'clean_' + x)

dims_short = dims.sort_values(by = 'pID', axis = 0)
dims_short = dims_short[['pID', '0','1','2','3']]
dims_short['group'] =  ['stress']*2446 +['pleasant']*4144+['unpleasant']*1016
dims_short = dims_short.reset_index(drop = True)
dims_short = dims_short.groupby(['pID', 'group']).agg(['min', 'max', 'mean'])
dims_short = dims_short.reset_index()

dims_short = pd.pivot_table(dims_short, index=['pID'], columns=['group'], aggfunc=np.sum, fill_value=0)
dims_short.columns = dims_short.columns.map('_'.join).str.strip('_')
dims_short = dims_short.reset_index()

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [237]:
X = consec.merge(dims_short, left_on = 'participant_id', right_on = 'pID', how = 'inner')
#X.columns = ['participant_id', 'pwvDmed', 'swvDmed', 'uwvDmed', 'pID','0_min','0_max','0_mean','1_min','1_max','1_mean',
#'2_min','2_max', '2_mean','3_min','3_max','3_mean']
X = X.merge(formal_vals, left_on = 'participant_id', right_on = 'pID', how = 'inner')
X = X[X.columns.drop(list(X.filter(regex='pID')))]
X = X.merge(consec_clean, left_on = 'participant_id', right_on = 'clean_participant_id', how = 'inner')
X = X.merge(shuffle, left_on = 'participant_id', right_on = 'participant_id', how = 'inner')
X = X.merge(shuffle_clean, left_on = 'participant_id', right_on = 'clean_participant_id', how = 'inner')


In [166]:
#aware merge will strip down to 120
aware = pd.read_csv('/users/madke/downloads/aware_personality_8-4.csv', index_col = 0)
awaregb = aware.groupby('participantID')


aware['localDate'] =pd.to_datetime(aware.localDate)
aware = aware.loc[:, aware.isna().mean() < .40]

sequences = []
for i in list(set(aware['participantID'])):
    unit = awaregb.get_group(i)
    unit = unit.sort_values(by='localDate')
    unit = unit.reset_index(drop = True)
    sequences.append(unit.reset_index(drop = False))


imp_seq = []
for i in sequences:
    needs_fixed = i.iloc[:,2:(len(i.columns)-10)]
    fixed = needs_fixed.fillna(needs_fixed.mean())
    fixed = fixed.fillna(0)
    new = pd.concat([i.iloc[:,0:2], fixed, i.iloc[:, (len(i.columns)-10):(len(i.columns))]], axis = 1)
    new = new.aggregate(func='mean', axis=0)
    imp_seq.append(new)
  

  
dep_vars = []
indep_vars = []
for i in imp_seq:
    dep = i.iloc[(len(i)-10):(len(i))]
    dep = pd.concat([dep, i[['participantID']]], axis = 0)
    dep_vars.append(dep)
    indep = i.iloc[0:len(i)-10].drop(['index'])
    indep_vars.append(indep)

In [167]:
X_aware = pd.concat(indep_vars, axis=1, keys=[s.name for s in indep_vars]).T.reset_index(drop = True)
X_aware = X_aware.loc[:, ~X_aware.columns.str.contains('^Unnamed')]
y = pd.concat(dep_vars, axis=1, keys=[s.name for s in dep_vars]).T.reset_index(drop = True)

In [191]:
X = X.merge(X_aware,left_on = 'participant_id', right_on = 'participantID', how = 'inner' )
y = y[y['participantID'].isin(list(X['participant_id']))]

In [238]:
X = X[X.columns.drop(list(X.filter(regex='participant')))]
X = X.fillna(0)

In [175]:
non_ys = y.drop(['neoNeuroticism'], axis = 1)
#X = pd.concat([X, non_ys], axis = 1)
y = y['neoNeuroticism']
y = y.reset_index(drop = True)

In [135]:
#for full (not aware merged)

df_measures = pd.read_csv('/users/madke/documents/AAPECSMadeline1.csv/AAPECSMadeline.csv')
skinnymeas_df = df_measures[['participantID', 'neoOpenness', 'neoConscientiousness', 'neoExtraversion',
                             'neoAgreeableness', 'neoNeuroticism', 'iipscDOM', 'iipscLOV', 'iipscELEV']]

skinnymeas_df = skinnymeas_df.groupby('participantID').agg(['mean'])
skinnymeas_df.columns = skinnymeas_df.columns.droplevel(1)

C:\Users\madke\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning:

Columns (95,111,115,116,123,133,135,142,143,155,159,166,167,175,179,186,190,198,203,207,208,210,212,214,216,221,232,233,242,247,248,249,253,257,258,263,282,285,289,293,295,296,303,307,309,324,329,333,334,337,338,339,340,343,344,347,351,353,358,360,365,367,371,372,377,383,387,388,389,393,394,395,397,401,402,408,409,413,414,415,419,422,428,429,431,435,437,441,442,444,446,447,448,450,452,453,457,458,459,461,462,464,478,484,494,497,503,504,508,509,511,512,514,515,518,519,520,522,523,524,525,526,527,530,531,538,539,540,542,543,545,550,556,560,562,564,568,570,573,574,576,579,580,581,583,584,585,586,587,588,589,590,591,592,593,594,595,597,715,717,718,720,725,726,727,728,730,731,732,733,734,736,738,739,740,742,743,744,745,746,750,752,753,754,755,756,758,759,760,761,762,764,765,766,767,768,769,772,773,775,776,777,778,779,780,781,783,784,785,786,787,788,791,792,793,794,796,797,798,799,800,801,803,804

In [263]:
y = skinnymeas_df['iipscDOM']
y = y.reset_index(drop = True)

In [264]:
best_param = specify_best(model = RandomForestRegressor(random_state = 15), typ = 'RF', X = X, y = y)

rf = RandomForestRegressor(min_samples_split=best_param.get('min_samples_split'), 
                           max_features= best_param.get('max_features'),random_state = 15)
rf_res = run_each_participant(rf, X, y, pi_bool= False, feat_bool = True)


{'max_features': 14, 'min_samples_split': 97}


C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launche

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launche

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launche

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launche

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launche

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launche

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\madke\Anaconda3\lib\site-packages\ipykernel_launche

In [262]:
#iipscelev
eprf = ext_featEPRF(rf_res[2], mean = True)
print_vals(best_param, rf_res, eprf)

mss: 87
max_feat: 8
mean_mae: 0.14103880693424783
mean_mse: 0.02910158234313638

frequency: 

284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284

feat_imp: 

0.019325774897544022
0.020795787437119337
0.027581078177569274
0.027776385187481472
0.028382656265953748
0.02840493698229888
0.02851274262246506
0.030835181957007518
0.032452589989843024
0.032836793007058676
0.03297585056478657
0.03366172085159389
0.0392155208600137
0.040125044556894626
0.05483466624624663
0.06615252369129888
0.06652075764101631
0.06902453157133556

top_ features

mec_stress
0_mean_stress
adc_pleasant
1_mean_pleasant
wc_pleasant
1_min_unpleasant
1_mean_stress
pwvDmed
clean_uwvVar
clean_swvDmed
1_min_stress
pwvVar
clean_uwvDmed
2_mean_stress
swvVar
clean_swvVar
wc_stress
swvDmed


In [265]:
#iipscdom
eprf = ext_featEPRF(rf_res[2], mean = True)
print_vals(best_param, rf_res, eprf)

mss: 97
max_feat: 14
mean_mae: 0.11618135335782023
mean_mse: 0.02207195863182459

frequency: 

284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284

feat_imp: 

0.01730976359411933
0.019652088140345542
0.019975818657440886
0.022064673628211212
0.027843889126614677
0.02909795545136543
0.02935336363689891
0.03062264039009545
0.031025341561474185
0.03161883492405303
0.03357283120309899
0.03465873033886355
0.03545278372783701
0.035517490748186886
0.03672156276651813
0.03696479910756436
0.04094895729244286
0.06355011753404277
0.08378898781093569
0.09322288497716702

top_ features

0_max_pleasant
1_mean_stress
2_mean_stress
2_min_stress
adc_stress
1_min_pleasant
3_max_stress
0_min_pleasant
3_min_stress
wc_pleasant
1_min_stress
2_max_pleasant
wc_unpleasant
adc_unpleasant
mec_stress
0_mean_stress
mec_unpleasant
wc_stress
clean_uwvVar
clean_uwvDmed


In [259]:
#iipsclov
eprf = ext_featEPRF(rf_res[2], mean = True)
print_vals(best_param, rf_res, eprf)

mss: 72
max_feat: 12
mean_mae: 0.12025772826304777
mean_mse: 0.025073335239920336

frequency: 

284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284

feat_imp: 

0.016284796609054727
0.016320857647379807
0.018993405745402616
0.019192148407841447
0.020247949853226865
0.02074282066558707
0.021620214563661273
0.024035079972535324
0.024315685905400508
0.025101437622308955
0.025935674473849805
0.027722990180179815
0.027936024656924605
0.028121582739385746
0.030495688049265467
0.031008410631548782
0.03496222758626157
0.03674261483402044
0.03729875486561723
0.04128041991110415
0.04170721926935534
0.06998427264401139
0.07006488885802425

top_ features

clean_swvDmed
1_min_pleasant
3_max_pleasant
mec_pleasant
clean_uwvVar
uwvDmed
pwvVar
pwvDmed
passive_pleasant
clean_pwvDmed
clean_swvVar
clean_pwvVar
1_max_stress
2_mean_pleasant
adc_pleasant
wc_pleasant
wc_stress
swvVar
1_mean_pleasant
swvDmed
0_min_pleasant
1_mean_stress
uwvVar


In [256]:

eprf = ext_featEPRF(rf_res[2], mean = True)
print_vals(best_param, rf_res, eprf)

mss: 12
max_feat: 2
mean_mae: 0.4859509144947107
mean_mse: 0.37809577226302743

frequency: 

284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284

feat_imp: 

0.016040244066232952
0.016049935311832807
0.016881761696636556
0.017093661377392017
0.01715426471671228
0.017481317608076228
0.018346923856221855
0.018681942366297128
0.019185083870561358
0.01929479915230107
0.019322615489297385
0.019609112167136893
0.01996039308330095
0.0202525677696491
0.020269151392270507
0.020500801925911676
0.02164758398859563
0.02204776777913575
0.022384066032878628
0.0224140412016551
0.02282761280650642
0.022844313862983346
0.023606110812688836
0.02368415494353799
0.023765279667025328
0.024224736503418297
0.02454908746269662
0.025244243261566473
0.02546897403824533
0.026126054569930803
0.026433653767622523
0.026681417287729554
0.026934906785642122
0.027254275784485416
0.027709099613583792

top_ features

2_mean_stress
2_

In [252]:
#neoExtraversion
eprf = ext_featEPRF(rf_res[2], mean = True)
print_vals(best_param, rf_res, eprf)

mss: 7
max_feat: 4
mean_mae: 0.4934869006196168
mean_mse: 0.38029725556230604

frequency: 

284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284

feat_imp: 

0.016325696830334926
0.01702192700922719
0.017027380934399913
0.01735515464061371
0.018202575698505554
0.018601597778963738
0.019270456485932085
0.019679136667133102
0.019710447301484614
0.019896404756354923
0.020346142497510194
0.020379579603428368
0.020879722081650655
0.021314984847207546
0.021707033101070753
0.021962040462920926
0.021966427078545292
0.022256386877982377
0.02239080484769683
0.02402866560297882
0.02421524063093484
0.02557110212171987
0.0264001008653886
0.02640316359090849
0.02684170497932616
0.028437888213213407
0.028440209585236288
0.028880276307323656
0.02931212793215324
0.029590461280828446
0.030397795705344503
0.03045828885191487

top_ features

2_mean_stress
passive_stress
2_max_pleasant
3_min_pleasant
clean_uwvVar
adc_unpleasant
3_ma

In [249]:
#neoAgreeableness
eprf = ext_featEPRF(rf_res[2], mean = True)
print_vals(best_param, rf_res, eprf)

mss: 92
max_feat: 6
mean_mae: 0.33497168898144386
mean_mse: 0.18922455557182977

frequency: 

284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284

feat_imp: 

0.016625276384416396
0.01683751632818611
0.017179547696296447
0.017920098091102104
0.01871681945244935
0.02034824611540425
0.021197008705975757
0.021756147097709493
0.02295704615115923
0.023793584567131765
0.02432724226557669
0.024983395080516092
0.030298750448677723
0.030928118385701562
0.03406588298072013
0.03443231214591612
0.03803283113943142
0.04096549408265125
0.0414706097624603
0.043979715585583966
0.0461956944772713
0.048183918615970996
0.04844114859548186
0.06315666721388445

top_ features

passive_unpleasant
pwvDmed
pwvVar
1_max_stress
1_mean_pleasant
0_min_pleasant
clean_swvVar
2_max_pleasant
1_max_pleasant
adc_pleasant
clean_swvDmed
mec_stress
wc_unpleasant
wc_stress
swvVar
2_mean_pleasant
wc_pleasant
1_mean_stress
clean_pwvVar
uwvVar
adc_unpleasant
adc_stress
clean_pwvDmed


In [246]:
#neoopenness
eprf = ext_featEPRF(rf_res[2], mean = True)
print_vals(best_param, rf_res, eprf)

mss: 17
max_feat: 4
mean_mae: 0.4327726119673897
mean_mse: 0.27789541261232514

frequency: 

284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284

feat_imp: 

0.016618966499124872
0.01709515225530304
0.017765744745370186
0.01779038448377168
0.01781889412902959
0.01822271565650732
0.018917286960300834
0.019925332155299656
0.020175222163293576
0.02062009679697983
0.02124624727323133
0.021506361063415077
0.02175079114703997
0.02203073018013475
0.022762104330907085
0.022813100490747872
0.02288403691122041
0.02404055749996627
0.025304248968205326
0.026811633038605124
0.027990273147465283
0.02815028133586415
0.028312525909353244
0.02925484997073532
0.030873546927485843
0.03242044728272037
0.033315754163400976
0.03437162734064867
0.04423148809912969

top_ features

mec_pleasant
1_mean_pleasant
uwvDmed
1_min_pleasant
0_min_pleasant
2_mean_stress
0_max_pleasant
swvDmed
wc_pleasant
0_mean_pleasant
2_mean_pleasant
2_min_pleasant
1_max

In [243]:
#neoNeuroticism
eprf = ext_featEPRF(rf_res[2], mean = True)
print_vals(best_param, rf_res, eprf)

mss: 12
max_feat: 8
mean_mae: 0.5318236627276294
mean_mse: 0.41679673588100663

frequency: 

284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284
284

feat_imp: 

0.01718619427120157
0.018185178336909486
0.01926161079452478
0.020275773754448705
0.020768270880891854
0.020800286413006807
0.020844604021802802
0.021035924682000866
0.021911413174650767
0.022041162673436986
0.022495152804669027
0.0227056005540317
0.023092519422933964
0.02438473938475607
0.025279353460511855
0.026810574726004604
0.03076019200889932
0.0334371272406829
0.03487524072980532
0.03539149999099961
0.03721826776696176
0.03760900928894605
0.05064327513219563
0.055051033917232046

top_ features

2_min_pleasant
clean_uwvVar
wc_unpleasant
mec_stress
adc_stress
clean_swvDmed
1_mean_pleasant
2_mean_pleasant
mec_pleasant
clean_pwvDmed
clean_pwvVar
0_min_pleasant
adc_unpleasant
clean_swvVar
clean_uwvDmed
1_mean_stress
adc_pleasant
wc_stress
pwvDmed
1_min_stress
pwvVar
wc_pleasant
swvDmed

In [217]:

totals = pd.read_csv('/users/madke/documents/030520_aapecstxtbtemapid.csv', index_col = 0)

In [266]:
X.columns

Index(['pwvDmed', 'swvDmed', 'uwvDmed', '0_max_pleasant', '0_max_stress',
       '0_max_unpleasant', '0_mean_pleasant', '0_mean_stress',
       '0_mean_unpleasant', '0_min_pleasant', '0_min_stress',
       '0_min_unpleasant', '1_max_pleasant', '1_max_stress',
       '1_max_unpleasant', '1_mean_pleasant', '1_mean_stress',
       '1_mean_unpleasant', '1_min_pleasant', '1_min_stress',
       '1_min_unpleasant', '2_max_pleasant', '2_max_stress',
       '2_max_unpleasant', '2_mean_pleasant', '2_mean_stress',
       '2_mean_unpleasant', '2_min_pleasant', '2_min_stress',
       '2_min_unpleasant', '3_max_pleasant', '3_max_stress',
       '3_max_unpleasant', '3_mean_pleasant', '3_mean_stress',
       '3_mean_unpleasant', '3_min_pleasant', '3_min_stress',
       '3_min_unpleasant', 'adc_stress', 'adc_pleasant', 'adc_unpleasant',
       'mec_stress', 'mec_pleasant', 'mec_unpleasant', 'passive_stress',
       'passive_pleasant', 'passive_unpleasant', 'val_stress', 'val_pleasant',
       'val_unpl